# Financial data analysis with Anthropic Claude 3 Sonnet in Amazon Bedrock

### Introduction
In this notebook, we will show you how to use Anthropic Claude 3 vision capabilities to prompt an image and get useful insights from it. 
We'll be utilizing Bedrock's Claude v3 Sonnet through the Boto3 API.
The prompt used in this example is called a zero-shot prompt because we are not providing any examples of text other than the prompt.

### Prerequisites
Run the cells in this section to install the packages needed by this notebook

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

### Setup
Add the import statements and the file and image processing helper functions.

In [2]:
import boto3
import json
import base64
from io import BytesIO

### Create a Class that can store image and text messages
<li> MAX_MESSAGES sets the upper limit for previous chat messages kept in memory. </li>

In [3]:
MAX_MESSAGES = 20

class ChatMessage():
    def __init__(self, role, message_type, text, bytesio=None):
        self.role = role
        self.message_type = message_type
        self.text = text
        self.bytesio = bytesio

### Define some helper functions
<li> These statements allow us to use the Boto3 library to call Bedrock, and process image data. </li>
<li> The helper functions are used to convert data between files, images, and base64-encoded bytes. </li>

In [4]:

#get a BytesIO object from file bytes
def get_bytesio_from_bytes(image_bytes):
    image_io = BytesIO(image_bytes)
    return image_io

#get a base64-encoded string from file bytes
def get_base64_from_bytes(image_bytes):
    resized_io = get_bytesio_from_bytes(image_bytes)
    img_str = base64.b64encode(resized_io.getvalue()).decode("utf-8")
    return img_str

#load the bytes from a file on disk
def get_bytes_from_file(file_path):
    with open(file_path, "rb") as image_file:
        file_bytes = image_file.read()
    return file_bytes
    

### Function to convert ChatMessages to the Claude 3 Messages API format.
<li> This format allows us to send a list of current and past messages to Claude 3 for processing. </li>

In [5]:
def convert_chat_messages_to_messages_api(chat_messages):
    
    messages = []
    
    for chat_msg in chat_messages:
        if (chat_msg.message_type == 'image'):
            messages.append({
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": chat_msg.text,
                        },
                    }
                ]
            })
        else:
            messages.append({
                "role": chat_msg.role,
                "content": [
                    {
                        "type": "text",
                        "text": chat_msg.text
                    }
                ]
            })
            
    return messages



### Request body builder function
<li> Function prepares the request payload for submission to Bedrock </li>
<li> Get the stringified request body for the InvokeModel API call </li>

In [6]:

def get_multimodal_chat_request_body(chat_messages):
    
    messages = convert_chat_messages_to_messages_api(chat_messages)
    
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 40000,
        "temperature": 0,
        "messages": messages,
    }
    
    return json.dumps(body)



### Function to generate a response using Anthropic Claude

In [7]:

def chat_with_model(message_history, new_text=None, new_image_bytes=None):
    session = boto3.Session()
    
    bedrock = session.client(service_name='bedrock-runtime') #creates a Bedrock client
    
    if new_text:
        new_text_message = ChatMessage('user', 'text', text=new_text)
        message_history.append(new_text_message)
        
    elif new_image_bytes:
        image_bytesio = get_bytesio_from_bytes(new_image_bytes)
        image_base64 = get_base64_from_bytes(new_image_bytes)
        new_image_message = ChatMessage('user', 'image', text=image_base64, bytesio=image_bytesio)
        message_history.append(new_image_message)
    
    
    number_of_messages = len(message_history)
    
    if number_of_messages > MAX_MESSAGES:
        del message_history[0 : (number_of_messages - MAX_MESSAGES) * 2] #make sure we remove both the user and assistant responses
    
    
    body = get_multimodal_chat_request_body(message_history)
    
    response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0", contentType="application/json", accept="application/json")
    
    response_body = json.loads(response.get('body').read()) # read the response
    
    output = response_body['content'][0]['text']
    
    response_message = ChatMessage('assistant', 'text', output)
    
    message_history.append(response_message)
    
    print(response_message.text)
    return response_message.text, message_history


### Read image data and get the bytes from file

##### Data used for reading the images are - Balance Sheet, Cash flow statement, Income Statement which are synthetic data and mocked up following financial statements format

In [8]:
file_bytes = get_bytes_from_file('./Balance Sheet.png')
file_bytes1 = get_bytes_from_file('./cash flow statement.png')
file_bytes2 = get_bytes_from_file('./income statement.png')

### Calling the Claude 3 model with the image bytes
<li> Create a string to store the response message (rm) </li>
<li> Create a list to store the message history (mh) </li>
<li> The model returns the response message and message history </li>
<li> Note: we are invoking the model with just the image bytes and not passing on any prompts to the model </li>

In [9]:
rm = "" #response message
mh = [] #message history
rm, mh = chat_with_model(mh, None, file_bytes)
rm, mh = chat_with_model(mh, None, file_bytes1)
rm, mh = chat_with_model(mh, None, file_bytes2)
# rm, mh = chat_with_model(mh, None, file_bytes3)

The image shows the balance sheet of Enterprise Industries as of December 31, 2022. The balance sheet lists the company's assets, liabilities, and shareholders' equity.

On the asset side, the current assets include cash and cash equivalents of $10,000, accounts receivable of $15,000, and inventory of $20,000, totaling $45,000 in current assets. The company also has property, plant, and equipment worth $80,000 and goodwill and intangible assets worth $25,000, bringing the total assets to $150,000.

On the liabilities side, the current liabilities include accounts payable of $10,000, accrued expenses of $5,000, and short-term debt of $5,000, totaling $20,000 in current liabilities. The company also has long-term debt of $40,000 and deferred tax liabilities of $5,000, bringing the total liabilities to $65,000.

The shareholders' equity section shows that the company has $20,000 in common stock, $10,000 in additional paid-in capital, and $55,000 in retained earnings, totaling $85,000 in s

In [10]:
prompt = """Perform a financial analysis by calculating the ratios from the data in the images. 
Interpret the ratios. 
Give your answer in Tabular format indifferent section with 3 columns - Ratios for that section, Ratio calculation, Interpretation for the ratio values. 
For every ratio give calculations/basis/justifications. 
Do not make up answers/numbers/data. 
Answer only if you know it.
Activity Ratios:

| Activity Ratios | Ratio calculation |
|-----------------|-------------------|
| Inventory turnover | Cost of goods sold / Average inventory |
| Days of inventory on hand (DOH) | Number of days in period / Inventory turnover |
| Receivables turnover | Revenue or Net credit sales / Average receivables |
| Days of sales outstanding (DSO) | Number of days in period / Receivables turnover |
| Payable Turnover | Purchases / Average payables |
| Number of days of payables | Number of days in period / Payable turnover |
| Working capital turnover | Revenue / Average working capital |
| Fixed assets turnover | Revenue / Average fixed assets |
| Total assets turnover | Revenue / Average total assets |
"""

In [11]:
rm, mh = chat_with_model(mh, prompt)

Activity Ratios:

| Activity Ratios | Ratio Calculation | Interpretation |
|-----------------|--------------------|--------------------|
| Inventory turnover | Cost of goods sold / Average inventory = $60,000 / ($20,000 + $20,000)/2 = $60,000 / $20,000 = 3 times | The inventory turned over 3 times during the year, indicating efficient inventory management. |
| Days of inventory on hand (DOH) | 365 days / Inventory turnover = 365 / 3 = 121.67 days | On average, inventory was held for approximately 122 days before being sold. |
| Receivables turnover | Revenue / Average receivables = $100,000 / ($15,000 + $15,000)/2 = $100,000 / $15,000 = 6.67 times | Accounts receivable turned over 6.67 times during the year, indicating efficient collection of receivables. |
| Days of sales outstanding (DSO) | 365 days / Receivables turnover = 365 / 6.67 = 54.72 days | On average, it took approximately 55 days to collect accounts receivable. |
| Payable Turnover | Cost of goods sold / Average payables =

In [12]:
prompt1 = """Perform a financial analysis by calculating the ratios from the data in the images. 
Interpret the ratios. 
Give your answer in Tabular format indifferent section with 3 columns - Ratios for that section, Ratio calculation, Interpretation for the ratio values. 
For every ratio give calculations/basis/justifications. 
Do not make up answers/numbers/data. 
Answer only if you know it.
Liquidity Ratios:

| Liquidity ratios | Ratio calculation |
|-----------------|-------------------|
| Current | Current assets / Current liabilities |
| Quick | (Cash + Short-term marketable securities + Receivables) / Current liabilities |
| Cash | (Cash + Short-term marketable securities) / Current liabilities |
| Defensive interval | (Cash + Short-term marketable securities + Receivables) / Daily expenditures |
| Cash conversion cycle | DOH + DSO - Number of days of payables |
"""

In [13]:
rm, mh = chat_with_model(mh, prompt1)

Liquidity Ratios:

| Liquidity Ratios | Ratio Calculation | Interpretation |
|-------------------|--------------------|--------------------|
| Current | Current assets / Current liabilities = $45,000 / $20,000 = 2.25 times | The company has $2.25 in current assets for every $1 of current liabilities, indicating a strong ability to meet short-term obligations. |
| Quick | (Cash + Receivables) / Current liabilities = ($10,000 + $15,000) / $20,000 = 1.25 times | The company has $1.25 in quick assets (cash and receivables) for every $1 of current liabilities, indicating a good level of liquidity. |
| Cash | (Cash) / Current liabilities = $10,000 / $20,000 = 0.5 times | The company has $0.50 in cash for every $1 of current liabilities, which may be a bit low for meeting immediate obligations. |
| Defensive Interval | (Cash + Receivables) / Daily expenditures = ($10,000 + $15,000) / ($60,000 / 365) = $25,000 / $164.38 = 152 days | The company has enough cash and receivables to cover approxim

In [14]:
prompt2 = """Perform a financial analysis by calculating the ratios from the data in the images. 
Interpret the ratios. 
Give your answer in Tabular format indifferent section with 3 columns - Ratios for that section, Ratio calculation, Interpretation for the ratio values. 
For every ratio give calculations/basis/justifications. 
Do not make up answers/numbers/data. 
Answer only if you know it.
Solvency Ratios:

| Solvency ratios | Ratio calculation |
|-----------------|-------------------|
| Debt-to-assets | Total debt / Total assets |
| Debt-to-capital | Total debt / (Total debt + Total shareholders' equity) |
| Debt-to-equity | Total debt / Total shareholders' equity |
| Financial leverage | Average total assets / Average shareholders' equity |
"""

In [15]:
rm, mh = chat_with_model(mh, prompt2)

Solvency Ratios:

| Solvency Ratios | Ratio Calculation | Interpretation |
|-----------------|--------------------|--------------------|
| Debt-to-assets | Total debt / Total assets = ($20,000 + $40,000) / $150,000 = 0.4 or 40% | The company has debt equal to 40% of its total assets, indicating a moderate level of debt relative to its asset base. |
| Debt-to-capital | Total debt / (Total debt + Total shareholders' equity) = $60,000 / ($60,000 + $85,000) = 0.414 or 41.4% | The company has debt equal to 41.4% of its total capital (debt plus equity), indicating a moderate level of debt relative to its capital structure. |
| Debt-to-equity | Total debt / Total shareholders' equity = $60,000 / $85,000 = 0.706 or 70.6% | The company has debt equal to 70.6% of its shareholders' equity, indicating a moderate level of debt relative to its equity. |
| Financial leverage | Average total assets / Average shareholders' equity = $150,000 / $85,000 = 1.765 | The company has $1.765 in assets for every

In [16]:
prompt3 = """Perform a financial analysis by calculating the ratios from the data in the images. 
Interpret the ratios. 
Give your answer in Tabular format indifferent section with 3 columns - Ratios for that section, Ratio calculation, Interpretation for the ratio values. 
For every ratio give calculations/basis/justifications. 
Do not make up answers/numbers/data. 
Answer only if you know it.
Coverage Ratios:

| Coverage Ratios | Ratio calculation |
|-----------------|-------------------|
| Interest coverage | EBIT / Interest expense |
| Fixed charge coverage | EBIT + Lease payments / (Interest expense + Lease payments) |
"""

In [17]:
rm, mh = chat_with_model(mh, prompt3)

Coverage Ratios:

| Coverage Ratios | Ratio Calculation | Interpretation |
|-----------------|--------------------|--------------------|
| Interest coverage | EBIT / Interest expense = ($20,000 + $500 - $2,000) / $2,000 = $18,500 / $2,000 = 9.25 times | The company's earnings before interest and taxes (EBIT) are 9.25 times higher than its interest expense, indicating a strong ability to cover interest payments from its operating income. |
| Fixed charge coverage | (EBIT + Lease payments) / (Interest expense + Lease payments) = Data not available | The information required to calculate the fixed charge coverage ratio, such as lease payments, is not provided in the financial statements. |

Note: The EBIT (Earnings Before Interest and Taxes) is calculated as Operating Income ($20,000) plus Interest Income ($500) minus Interest Expense ($2,000). Since there is no information provided about lease payments, the fixed charge coverage ratio cannot be calculated.


In [18]:
prompt4 = """Perform a financial analysis by calculating the ratios from the data in the images. 
Interpret the ratios. 
Give your answer in Tabular format indifferent section with 3 columns - Ratios for that section, Ratio calculation, Interpretation for the ratio values. 
For every ratio give calculations/basis/justifications. 
Do not make up answers/numbers/data. 
Answer only if you know it.
Profitability Ratios:

| Return on sales ratios | Ratio calculation |
|------------------------|-------------------|
| Gross profit margin | Gross profit / Revenue |
| Operating margin | Operating profit / Revenue |
| Pretax margin | EBT (Earnings before taxes) / Revenue |
| Net profit margin | Net income / Revenue |
"""

In [19]:
rm, mh = chat_with_model(mh, prompt4)

Profitability Ratios:

| Return on Sales Ratios | Ratio Calculation | Interpretation |
|------------------------|--------------------|--------------------|
| Gross Profit Margin | Gross Profit / Revenue = $40,000 / $100,000 = 0.4 or 40% | The company earns a gross profit of 40 cents for every dollar of revenue, indicating a healthy gross profit margin. |
| Operating Margin | Operating Profit / Revenue = $20,000 / $100,000 = 0.2 or 20% | The company earns an operating profit of 20 cents for every dollar of revenue, indicating a good level of profitability from operations. |
| Pretax Margin | EBT (Earnings Before Taxes) / Revenue = $18,500 / $100,000 = 0.185 or 18.5% | The company earns a pretax profit of 18.5 cents for every dollar of revenue, indicating a good level of profitability before taxes. |
| Net Profit Margin | Net Income / Revenue = $14,000 / $100,000 = 0.14 or 14% | The company earns a net profit of 14 cents for every dollar of revenue, indicating a reasonable level of profi

In [20]:
prompt5 = """Perform a financial analysis by calculating the ratios from the data in the images. 
Interpret the ratios. 
Give your answer in Tabular format indifferent section with 3 columns - Ratios for that section, Ratio calculation, Interpretation for the ratio values. 
For every ratio give calculations/basis/justifications. 
Do not make up answers/numbers/data. 
Answer only if you know it.
| Return on investment ratios | Ratio calculation |
|----------------------------|-------------------|
| Operating ROA | Operating profit / Average total assets |
| ROA | Net income / Average total assets |
| Return on total capital | EBIT / (Debt + Equity) |
| ROE | Net income / Average total equity |
| Return on common equity | Net income - Preferred dividends / Average common equity |
"""

In [21]:
rm, mh = chat_with_model(mh, prompt5)

Return on Investment Ratios:

| Return on Investment Ratios | Ratio Calculation | Interpretation |
|-----------------------------|--------------------|--------------------|
| Operating ROA | Operating Profit / Average Total Assets = $20,000 / $150,000 = 0.133 or 13.3% | The company generates an operating profit of 13.3 cents for every dollar of total assets, indicating a reasonable level of profitability from its assets. |
| ROA | Net Income / Average Total Assets = $14,000 / $150,000 = 0.093 or 9.3% | The company generates a net income of 9.3 cents for every dollar of total assets, indicating a moderate level of profitability from its assets. |
| Return on Total Capital | EBIT / (Debt + Equity) = $18,500 / ($60,000 + $85,000) = $18,500 / $145,000 = 0.128 or 12.8% | The company generates an EBIT of 12.8 cents for every dollar of total capital (debt plus equity), indicating a reasonable level of profitability from its capital. |
| ROE | Net Income / Average Total Equity = $14,000 / $85,

In [22]:
prompt6 = """Perform a financial analysis by calculating the ratios from the data in the images. 
Interpret the ratios. 
Give your answer in Tabular format indifferent section with 3 columns - Ratios for that section, Ratio calculation, Interpretation for the ratio values. 
For every ratio give calculations/basis/justifications. 
Do not make up answers/numbers/data. 
Answer only if you know it.
Valuation Ratios:

| Valuation ratios | Ratio calculation |
|------------------|-------------------|
| P/E | Stock price / Earnings per share |
| PCP | Stock price / Cash flow per share |
| PS | Stock price / Sales per share |
| PBV | Book per share / Average book value per share |
"""

In [23]:
rm, mh = chat_with_model(mh, prompt6)

Unfortunately, the financial statements provided do not contain information about the company's stock price, earnings per share, cash flow per share, sales per share, or book value per share. Without this data, it is not possible to calculate or interpret valuation ratios such as P/E, PCP, PS, or PBV.

Valuation ratios require market data (stock price) and per-share data (earnings per share, cash flow per share, sales per share, book value per share) that are typically not found in basic financial statements like the ones provided. These ratios are used by investors and analysts to assess the relative value of a company's stock compared to its financial performance and book value.

Unless additional information is provided, I cannot calculate or interpret any valuation ratios for this company based on the data available in the balance sheet, income statement, and cash flow statement.


In [24]:
prompt7 = """Perform a financial analysis by calculating the ratios from the data in the images. 
Interpret the ratios. 
Give your answer in Tabular format indifferent section with 3 columns - Ratios for that section, Ratio calculation, Interpretation for the ratio values. 
For every ratio give calculations/basis/justifications. 
Do not make up answers/numbers/data. 
Answer only if you know it.
| Price per share | Ratio calculation |
|-----------------|-------------------|
| Basic EPS | (Net income - Preferred dividends) / Weighted average number of ordinary shares outstanding |
| Diluted EPS | (Net income - Preferred dividends) / Weighted number of ordinary shares outstanding + Potential dilutive effects of options, warrants, convertible securities that have been issued at conversion |
| Cash flow per share | Cash flow from operations / Weighted average number of ordinary shares outstanding |
| EBITDA per share | EBITDA / Average number of common stock |
| Dividends per share | Dividends paid / Number of shares outstanding |

"""

In [25]:
rm, mh = chat_with_model(mh, prompt7)

Unfortunately, the financial information provided in the income statement, balance sheet, and cash flow statement does not include details such as preferred dividends, weighted average number of ordinary shares outstanding, potential dilutive effects of options/warrants/convertibles, EBITDA, or dividends paid. Without this additional data, it is not possible to calculate or interpret ratios related to price per share, earnings per share, cash flow per share, EBITDA per share, or dividends per share.

These ratios require specific information about a company's share capital structure, outstanding shares, dilutive securities, and dividend payments, which are typically not found in basic financial statements. They are used by investors and analysts to assess the value of a company's shares and the potential returns to shareholders.

Unless additional information is provided regarding the company's share capital and dividend details, I cannot calculate or interpret any ratios related to pr

In [26]:
prompt8 = """Perform a financial analysis by calculating the ratios from the data in the images. 
Interpret the ratios. 
Give your answer in Tabular format indifferent section with 3 columns - Ratios for that section, Ratio calculation, Interpretation for the ratio values. 
For every ratio give calculations/basis/justifications. 
Do not make up answers/numbers/data. 
Answer only if you know it.
| Dividend-related ratios | Ratio calculation |
|-------------------------|-------------------|
| Dividend payout ratio | Common share dividends / Net income attributable to common shares |
| Retention rate (k) | Net income attributable to common shares - Common share dividends / Net income attributable to common shares |
| Sustainable growth rate | b x ROE |
"""

In [27]:
rm, mh = chat_with_model(mh, prompt8)

Unfortunately, the financial statements provided do not include information about common share dividends, net income attributable to common shares, or the retention rate (b). Without this data, it is not possible to calculate or interpret dividend-related ratios such as the dividend payout ratio, retention rate (k), or sustainable growth rate.

These ratios require specific details about a company's dividend payments, net income allocated to common shareholders, and the retention rate of earnings, which are typically not found in basic financial statements like the ones provided.

Unless additional information is provided regarding the company's dividend payments, net income attributable to common shares, and retention rate, I cannot calculate or interpret any dividend-related ratios based on the data available in the financial statements.


In [28]:
prompt9 = """Perform a financial analysis by calculating the ratios from the data in the images. 
Interpret the ratios. 
Give your answer in Tabular format indifferent section with 3 columns - Ratios for that section, Ratio calculation, Interpretation for the ratio values. 
For every ratio give calculations/basis/justifications. 
Do not make up answers/numbers/data. 
Answer only if you know it.
Solvency Ratios:

| Ratio per share | Ratio calculation |
|-----------------|-------------------|
| EBIT interest coverage | EBIT / Gross interest prior to deductions for capitalized interest or interest income |
| EBITDA interest coverage | EBITDA / Gross interest prior to deductions for capitalized interest or interest income |
"""

In [29]:
rm, mh = chat_with_model(mh, prompt9)

Unfortunately, the financial statements provided do not include information about gross interest prior to deductions for capitalized interest or interest income, or EBITDA (Earnings Before Interest, Taxes, Depreciation, and Amortization). Without this data, it is not possible to calculate or interpret the EBIT interest coverage ratio or the EBITDA interest coverage ratio.

These ratios are used to assess a company's ability to cover its interest expenses from its operating income and cash flows. They require specific details about gross interest expenses and EBITDA, which are typically not found in basic financial statements like the ones provided.

Unless additional information is provided regarding the company's gross interest expenses and EBITDA, I cannot calculate or interpret the EBIT interest coverage ratio or the EBITDA interest coverage ratio based on the data available in the financial statements.


In [30]:
prompt10 = """Perform a financial analysis by calculating the ratios from the data in the images. 
Interpret the ratios. 
Give your answer in Tabular format indifferent section with 3 columns - Ratios for that section, Ratio calculation, Interpretation for the ratio values. 
For every ratio give calculations/basis/justifications. 
Do not make up answers/numbers/data. 
Answer only if you know it.
Price per Share:

| Price per share | Ratio calculation |
|-----------------|-------------------|
| FFO (Funds from operations) divided by debt | FFO + Interest paid - Operating lease adjustments - Gains/losses from property sales + Straight-line interest or implied income |
| Return on capital | EBIT / Average capital |
| FFO (Funds from operations) to debt | FFO / Total debt |
| Free operating cash flow to debt | CFO (Adjusted) - Capital expenditures - Dividends paid / Total debt |
| Discretionary cash flow to debt | CFO - Capital expenditures - Dividend paid / Total debt |
| Net cash flow-to-capital expenditures | FFO - Dividends / Capital expenditures |
"""

In [31]:
rm, mh = chat_with_model(mh, prompt10)

Unfortunately, the financial statements provided do not include information about funds from operations (FFO), interest paid, operating lease adjustments, gains/losses from property sales, straight-line interest or implied income, adjusted cash flow from operations (CFO), capital expenditures, or dividends paid. Without this additional data, it is not possible to calculate or interpret the ratios related to price per share, return on capital, FFO to debt, free operating cash flow to debt, discretionary cash flow to debt, or net cash flow to capital expenditures.

These ratios are commonly used in the analysis of real estate investment trusts (REITs) or companies with significant real estate holdings. They require specific details about funds from operations, cash flows, capital expenditures, debt, and dividends, which are typically not found in basic financial statements like the ones provided.

Unless additional information is provided regarding the company's funds from operations, ca

In [32]:
prompt11 = """Perform a financial analysis by calculating the ratios from the data in the images. 
Interpret the ratios. 
Give your answer in Tabular format indifferent section with 3 columns - Ratios for that section, Ratio calculation, Interpretation for the ratio values. 
For every ratio give calculations/basis/justifications. 
Do not make up answers/numbers/data. 
Answer only if you know it.
Leverage Ratios:

| Leverage Ratios |
|-----------------|
| Debt-to-assets ratio |
| Debt-to-capital ratio |
| Debt-to-equity ratio |
| Financial leverage ratio |

* Debt is defined as the sum of interest-bearing short-term and long-term debt.
"""




In [33]:
rm, mh = chat_with_model(mh, prompt11)

Leverage Ratios:

| Leverage Ratios | Ratio Calculation | Interpretation |
|-----------------|--------------------|--------------------|
| Debt-to-assets ratio | Total Debt / Total Assets = ($20,000 + $40,000) / $150,000 = 0.4 or 40% | The company has debt equal to 40% of its total assets, indicating a moderate level of leverage. |
| Debt-to-capital ratio | Total Debt / (Total Debt + Total Shareholders' Equity) = $60,000 / ($60,000 + $85,000) = 0.414 or 41.4% | The company has debt equal to 41.4% of its total capital (debt plus equity), indicating a moderate level of leverage relative to its capital structure. |
| Debt-to-equity ratio | Total Debt / Total Shareholders' Equity = $60,000 / $85,000 = 0.706 or 70.6% | The company has debt equal to 70.6% of its shareholders' equity, indicating a moderate level of leverage relative to its equity. |
| Financial leverage ratio | Average Total Assets / Average Shareholders' Equity = $150,000 / $85,000 = 1.765 | The company has $1.765 in assets 

In [34]:
prompt12 = """Perform a financial analysis by calculating the ratios from the data in the images. 
Interpret the ratios. 
Give your answer in Tabular format indifferent section with 3 columns - Ratios for that section, Ratio calculation, Interpretation for the ratio values. 
For every ratio give calculations/basis/justifications. 
Do not make up answers/numbers/data. 
Answer only if you know it.
Segment Ratios:

| Segment ratio | Numerator | Denominator | Indication |
|---------------|------------|--------------|-------------|
| Segment margin | Segment profit (loss) | Segment revenue | Measures a segment's profitability relative to its revenue. |
| Segment turnover | Segment revenue | Segment assets | Measures a segment's efficiency in using its assets. |
| Segment ROA | Segment profit (loss) | Segment assets | Measures a segment's profitability relative to its assets. |
| Segments' sales ratio | Segment liabilities | Segment assets | Measures a segment's leverage. |

"""




In [35]:
rm, mh = chat_with_model(mh, prompt12)

Unfortunately, the financial statements provided do not include any segmented information or breakdowns by business segments. Without segment-level data on revenue, profit/loss, assets, and liabilities for different operating segments of the company, it is not possible to calculate or interpret segment ratios such as segment margin, segment turnover, segment ROA, or segments' sales ratio.

These ratios are used to analyze the performance, efficiency, profitability, and leverage of a company's individual business segments or operating divisions. They require detailed financial information to be reported separately for each segment, which is typically not found in basic consolidated financial statements like the ones provided.

Unless additional segmented financial data is provided for the company's different business segments or operating divisions, I cannot calculate or interpret any segment ratios based on the information available in the consolidated financial statements.


In [36]:
prompt13 = """Perform a financial analysis by calculating the ratios from the data in the images. 
Interpret the ratios. 
Give your answer in Tabular format indifferent section with 3 columns - Ratios for that section, Ratio calculation, Interpretation for the ratio values. 
For every ratio give calculations/basis/justifications. 
Do not make up answers/numbers/data. 
Answer only if you know it.
Performance Ratios:

| Leverage Ratios | Calculation | Indication |
|-----------------|--------------|------------|
| Cash flow to income | CFO / Net income | Operating cash generated per dollar of net income |
| Cash return on assets | CFO / Average total assets | Operating cash generated per dollar of assets investment |
| Cash return on equity | CFO / Average shareholders' equity | Operating cash generated per dollar of owners investment |
| Cash to income | CFO / Operating income | Cash generated from operations |
| Cash flow per share | (CFO - Dividends) / Number of common shares outstanding | Operating cash flow on a per share basis |
| Debt payment | CFO / Long-term debt | Ability to pay debts with operating cash flows |
| Dividend payment | CFO / Dividends paid | Ability to pay dividends with operating cash flows |
| Investing and Financing | CFO / (Capital expenditures + Financing activities) | Ability to acquire assets, pay debts, and pay dividends to owners |
| Debt coverage | CFO / Total debt | Financial risk and financial leverage |
| Interest Coverage | (CFO + Interest paid + Taxes paid) / Interest expense | Ability to meet interest obligations |
| Deinvestment | CFO / (Cash paid for long-term assets) | Ability to acquire assets with operating cash flows |
"""




In [37]:
rm, mh = chat_with_model(mh, prompt13)

Unfortunately, the financial statements provided do not include some key information required to calculate many of the performance ratios listed, such as:

- Cash Flow from Operations (CFO)
- Interest paid
- Taxes paid 
- Dividends paid
- Number of common shares outstanding
- Capital expenditures
- Financing activities
- Cash paid for long-term assets

Without this additional data, it is not possible to accurately calculate or interpret ratios such as cash flow to income, cash return on assets/equity, cash to income, cash flow per share, debt payment, dividend payment, investing and financing, debt coverage, interest coverage, or deinvestment.

These performance ratios are used to assess a company's ability to generate cash flows from operations, meet its debt and interest obligations, pay dividends, and fund investments and financing activities. They require specific details about cash flows, interest expenses, taxes, dividends, capital expenditures, and financing activities, which ar

In [38]:
prompt14 = """You are acting as a financial analyst. Can you summarize all the findings from previous prompts? Highlight the key takeaways."""


In [39]:
rm, mh = chat_with_model(mh, prompt14)

Based on the financial statements provided, here are the key takeaways and findings from the analysis:

1. Profitability Ratios:
   - The company has a reasonable operating profit margin (Operating ROA of 13.3%) and a moderate net profit margin (ROA of 9.3%), indicating moderate profitability from its assets.
   - The return on total capital (12.8%) and return on equity (16.5%) suggest a reasonable level of profitability from the company's capital and shareholders' equity.

2. Leverage Ratios:
   - The company has a moderate level of leverage, with a debt-to-assets ratio of 40%, a debt-to-capital ratio of 41.4%, and a debt-to-equity ratio of 70.6%.
   - The financial leverage ratio of 1.765 indicates a moderate level of financial leverage, with $1.765 in assets for every $1 of shareholders' equity.

3. Limitations:
   - Due to the lack of specific information, it was not possible to calculate or interpret valuation ratios, dividend-related ratios, solvency ratios, price per share ratio